# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [123]:
pip install cvxpy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [124]:
pip install yfinance



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [125]:
# YOUR CHANGES HERE

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf


tickers = ["AAPL", "AMZN", "GOOG", "META", "SPY", "TSLA"]

data = yf.download(tickers, period="2y")["Close"]

historical_prices = data.resample("M").last()

historical_prices.index = historical_prices.index.to_period("M")

historical_prices


/tmp/ipykernel_18445/3647220460.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="2y")["Close"]
[*********************100%***********************]  6 of 6 completed
/tmp/ipykernel_18445/3647220460.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  historical_prices = data.resample("M").last()


Ticker,AAPL,AMZN,GOOG,META,SPY,TSLA
Date,,,,,,
2023-09,169.549271,127.120003,131.070908,298.572021,417.865631,250.220001
2023-10,169.113556,133.089996,124.559601,299.626251,408.794373,200.839996
2023-11,188.355331,146.089996,133.128647,325.365021,446.135193,240.080002
2023-12,190.913651,151.940002,140.097229,352.028748,466.503632,248.479996
2024-01,182.851913,155.199997,140.962097,388.011353,473.933441,187.289993
2024-02,179.461273,176.759995,138.954041,487.972992,498.666534,201.880005
2024-03,170.257339,180.380005,151.360275,483.442993,514.973938,175.789993
2024-04,169.115570,175.000000,163.667130,428.276917,494.210358,183.279999
2024-05,191.138153,176.440002,172.932068,464.775482,519.207275,178.080002


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


Submit "historical_prices.tsv" in Gradescope.

In [126]:
# YOUR CHANGES HERE

import yfinance as yf
tickers = ["AAPL", "AMZN", "GOOG", "META", "SPY", "TSLA"]

data = yf.download(tickers, period="2y", auto_adjust=True)["Close"]

monthly_prices = data.resample("M").last().tail(24)

monthly_prices = monthly_prices.reset_index()

monthly_prices.rename(columns={"Date": "date"}, inplace=True)

monthly_prices.to_csv("historical_prices.tsv", sep="\t", index=False)



[*********************100%***********************]  6 of 6 completed
/tmp/ipykernel_18445/1116743526.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_prices = data.resample("M").last().tail(24)


## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [127]:
# YOUR CHANGES HERE

historical_returns = historical_prices.pct_change().dropna()
estimated_returns = historical_returns.mean().rename("return")
estimated_returns.sort_values()

Ticker
AAPL    0.016154
SPY     0.018989
TSLA    0.026376
GOOG    0.027039
AMZN    0.027498
META    0.043371
Name: return, dtype: float64

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [128]:
# YOUR CHANGES HERE
historical_returns_with_date = historical_returns.reset_index()
historical_returns_with_date.rename(columns={"Date": "date"}, inplace=True)
historical_returns_with_date.to_csv("historical_returns.tsv", sep="\t", index=False)

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [129]:
# YOUR CHANGES HERE

expected_returns = historical_returns.mean()


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [130]:
# YOUR CHANGES HERE

expected_returns_df = expected_returns.reset_index()
expected_returns_df.columns = ["asset", "estimated_return"]
expected_returns_df.to_csv("estimated_returns.tsv", sep="\t", index=False)

Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [131]:
# YOUR CHANGES HERE

covariance_matrix = historical_returns.cov()

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [132]:
# YOUR CHANGES HERE

covariance_matrix.to_csv("estimated_covariance.tsv", sep="\t")

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [133]:
# YOUR CHANGES HERE

import cvxpy as cp

# Number of assets
n = len(expected_returns)

# Decision variable: portfolio weights
w = cp.Variable(n)

# Objective: maximize expected return
objective = cp.Maximize(expected_returns.values @ w)

# Constraints: weights sum to 1 and non-negative
constraints = [
    cp.sum(w) == 1,
    w >= 0
]

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Check if solved successfully
if problem.status == 'optimal':
    max_return_weights = w.value
    print(f"Optimization successful, weights sum: {sum(max_return_weights)}")
else:
    print(f"Optimization failed with status: {problem.status}")

Optimization successful, weights sum: 0.9999999999999999


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [134]:
# YOUR CHANGES HERE

max_return_df = pd.DataFrame({
   'asset': expected_returns.index,
   'allocation': max_return_weights
})
max_return_df.to_csv("maximum_return.tsv", sep="\t", index=False)

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [135]:
# YOUR CHANGES HERE

import cvxpy as cp

n = len(expected_returns)

w = cp.Variable(n)

objective = cp.Minimize(cp.quad_form(w, covariance_matrix.values))

constraints = [cp.sum(w) == 1]

problem = cp.Problem(objective, constraints)
problem.solve()

min_risk_weights = w.value

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [136]:
# YOUR CHANGES HERE

min_risk_df = pd.DataFrame({
   'asset': expected_returns.index,
   'allocation': min_risk_weights
})
min_risk_df.to_csv("minimum_risk.tsv", sep="\t", index=False)

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [137]:
# YOUR CHANGES HERE

import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

estimated_return_minimum_risk = expected_returns.values @ min_risk_weights
estimated_return_maximum_return = expected_returns.values @ max_return_weights

n = len(expected_returns)

ef_variances = []
ef_returns = []
ef_portfolios = []

for r in np.linspace(estimated_return_minimum_risk, estimated_return_maximum_return, 101):
   x_r = cp.Variable(n)
   
   prob_r = cp.Problem(cp.Minimize(x_r.T @ covariance_matrix.values @ x_r),
                      [0 <= x_r, cp.sum(x_r) == 1, x_r.T @ expected_returns.values == r])
   
   ef_variances.append(prob_r.solve())
   ef_returns.append(r)
   ef_portfolios.append(x_r.value)

efficient_portfolios = np.array(ef_portfolios)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (101,) + inhomogeneous part.

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [ ]:
# YOUR CHANGES HERE

ef_data = {
   'index': range(101),
   'return': ef_returns,
   'risk': np.sqrt(ef_variances)
}

for i, ticker in enumerate(expected_returns.index):
   ef_data[ticker] = [portfolio[i] if portfolio is not None else 0 for portfolio in ef_portfolios]

ef_df = pd.DataFrame(ef_data)
ef_df.to_csv("efficient_frontier.tsv", sep="\t", index=False)

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [ ]:
# YOUR CHANGES HERE

np.random.seed(42)

# Storage for results
highest_return_assets = []

for i in range(1000):
   resampled_returns = np.random.multivariate_normal(
       mean=expected_returns.values, 
       cov=covariance_matrix.values, 
       size=23
   )
   
   resampled_mean_returns = np.mean(resampled_returns, axis=0)
   
   highest_return_index = np.argmax(resampled_mean_returns)
   highest_return_asset = expected_returns.index[highest_return_index]
   highest_return_assets.append(highest_return_asset)

from collections import Counter
asset_frequencies = Counter(highest_return_assets)
print("Asset selection frequency:")
for asset, count in asset_frequencies.items():
   print(f"{asset}: {count/1000:.3f} ({count}/1000)")

Asset selection frequency:
META: 0.572 (572/1000)
GOOG: 0.107 (107/1000)
TSLA: 0.261 (261/1000)
AMZN: 0.007 (7/1000)
SPY: 0.004 (4/1000)
AAPL: 0.049 (49/1000)


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [ ]:
# YOUR CHANGES HERE

total_samples = 1000
probabilities_data = []

for asset in expected_returns.index:
   count = asset_frequencies.get(asset, 0)
   probability = count / total_samples
   probabilities_data.append({'asset': asset, 'probability': probability})

prob_df = pd.DataFrame(probabilities_data)
prob_df.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

In [ ]:
with open("acknowledgements.txt", "w") as f:
    f.write("none")

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.